In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image
import pickle
from sklearn.utils import shuffle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import History 
from tensorflow.keras import backend as K
if K.backend() == 'tensorflow':
    import tensorflow as tf

In [ ]:
train_path = './seg_train/seg_train/'
test_path = './seg_test/seg_test/'
Labels = ['buildings', 'forest','glacier','mountain','sea','street']
label_dict = {0:'buildings', 1:'forest', 2:'glacier', 3:'mountain', 4:'sea', 5:'street'}

In [ ]:
min_count = len(os.listdir(train_path+'buildings'))*3

In [ ]:
def get_data(path):
    #train_label_count = {'buildings':0, 'forest':0, 'glacier':0, 'mountain':0, 'sea':0, 'street':0}
    images = list()
    lbls = list()
    for label in Labels:
        count = 0
        for image in os.listdir(path+label):
            if count < min_count:
                img = cv2.imread(path+label+r'/'+image)
                img = Image.fromarray(img , 'RGB')
                img = img.resize((50,50))
                images.append(np.array(img)/255)
                if label == 'buildings':
                    lbl = 0
                elif label == 'forest':
                    lbl = 1
                elif label == 'glacier':
                    lbl = 2
                elif label == 'mountain':
                    lbl = 3
                elif label == 'sea':
                    lbl = 4
                else:
                    lbl = 5
                lbls.append(lbl)
                if path == train_path:
                    rotated = img.rotate(45)
                    flipped = np.fliplr(img)
                    images.append(np.array(rotated)/255)
                    images.append(np.array(flipped)/255)
                    lbls.append(lbl)
                    lbls.append(lbl)
                    #train_label_count[label] += 3
                    count += 3
                
    images = np.array(images)
    lbls = np.array(lbls)
    
    return shuffle(images, lbls, random_state=42)

In [ ]:
train_images, train_labels = get_data(train_path)

In [ ]:
test_images, test_labels = get_data(test_path)

In [ ]:
model=Sequential()
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=train_images.shape[1:]))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(6,activation="softmax"))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
monitor = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

In [ ]:
model_h = model.fit(train_images,train_labels,epochs=50,validation_split=.25, callbacks=[monitor])

In [ ]:
model.evaluate(test_images,test_labels, verbose=1)